In [23]:
import datetime as dt
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf

## This project involves implementing a trading analysis strategy using Exponential Moving Averages (EMAs). The goal is to identify potential buy and sell signals based on the relationship between different EMAs and stock price movements. You can input any ticker in the input below, but for this project I used Microsoft.

In [24]:
yf.pdr_override()

stock = input('Enter a stock ticker symbol :')
print(stock)

MSFT


In [25]:
startdate = dt.datetime(2022,1,1)
enddate = dt.datetime.now()

In [26]:
df = pdr.get_data_yahoo(stock, startdate, enddate)

[*********************100%***********************]  1 of 1 completed


In [27]:
print(df)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-01-03  335.350006  338.000000  329.779999  334.750000  330.092255   
2022-01-04  334.829987  335.200012  326.119995  329.010010  324.432098   
2022-01-05  325.859985  326.070007  315.980011  316.380005  311.977875   
2022-01-06  313.149994  318.700012  311.489990  313.880005  309.512634   
2022-01-07  314.149994  316.500000  310.089996  314.040009  309.670441   
...                ...         ...         ...         ...         ...   
2023-06-27  331.859985  336.149994  329.299988  334.570007  334.570007   
2023-06-28  334.660004  337.980011  333.809998  335.850006  335.850006   
2023-06-29  334.709991  336.109985  332.619995  335.049988  335.049988   
2023-06-30  337.750000  342.730011  337.200012  340.540009  340.540009   
2023-07-03  339.190002  340.899994  336.570007  337.989990  337.989990   

              Volume  
Date          

In [28]:
emasUsed = [3,5,8,10,12,15,30,35,40,45,50,60]
for num in emasUsed:
    ema = num
    df['Ema_'+str(ema)] = round(df.iloc[:,4].ewm(span=ema, adjust=False).mean(),2)

print(df.tail())


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-06-27  331.859985  336.149994  329.299988  334.570007  334.570007   
2023-06-28  334.660004  337.980011  333.809998  335.850006  335.850006   
2023-06-29  334.709991  336.109985  332.619995  335.049988  335.049988   
2023-06-30  337.750000  342.730011  337.200012  340.540009  340.540009   
2023-07-03  339.190002  340.899994  336.570007  337.989990  337.989990   

              Volume   Ema_3   Ema_5   Ema_8  Ema_10  Ema_12  Ema_15  Ema_30  \
Date                                                                           
2023-06-27  24354100  333.59  334.33  334.78  334.67  334.33  333.53  327.14   
2023-06-28  20259500  334.72  334.84  335.02  334.88  334.56  333.82  327.70   
2023-06-29  16997000  334.89  334.91  335.03  334.91  334.64  333.97  328.18   
2023-06-30  26823800  337.71  336.79  336.25  335.94  335.55  334.79  328.98   
2

In [29]:
pos = 0
num = 0
percentchange=[]


for i in df.index:
    cmin = min(df['Ema_3'][i],df['Ema_5'][i],df['Ema_8'][i],df['Ema_10'][i],df['Ema_12'][i],df['Ema_15'][i])
    cmax = max(df['Ema_30'][i],df['Ema_35'][i],df['Ema_40'][i],df['Ema_45'][i],df['Ema_50'][i],df['Ema_60'][i])

    close = df['Adj Close'][i]

    if(cmin>cmax):
        print('Red White Blue')
        if(pos==0):
            bp =close
            pos=1
            print('Buying now at '+ str(bp))

    elif(cmin<cmax):
        print('Blue White Red')
        if(pos==1):
            sp = close
            pos= 0
            print('Selling now at '+ str(sp))
            pc= (sp/bp - 1) * 100
            percentchange.append(pc)
    if(num==df['Adj Close'].count()-1 and pos == 1):
        pos = 0
        sp = close
        pos= 0
        print('Selling now at '+ str(sp))
        pc= (sp/bp - 1) * 100
        percentchange.append(pc) 
    num+=1

print(pc)

Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Red White Blue
Buying now at 305.7456359863281
Red White Blue
Red White Blue
Red Whit

In [30]:
gains = 0
ng = 0
losses = 0
nl = 0
totalR = 1

for i in percentchange:
    if(i>0):
        gains+=i
        ng+=1
    else:
        losses+=i
        nl+=1
    totalR = totalR*(i/100)+1

totalR=round((totalR-1)*100,2)


if(ng>0):
    avgGain=gains/ng
    maxR = str(max(percentchange))
else:
    avgGain = 0
    maxR = 'undefined'

if(nl>0):
    avgLoss=losses/nl
    maxL = str(max(percentchange))
    ratio = str(-avgGain/avgLoss)
else:
    avgLoss = 0
    maxL = 'undefined'
    ratio = 'inf'

if(ng>0 or nl>0):
    battingAVG = ng/(ng+nl)
else:
    battingAVG = 0

In [31]:
print()
print("Results for "+ stock +" going back to "+str(df.index[0])+", Sample size: "+str(ng+nl)+" trades")
print("EMAs used: "+str(emasUsed))
print("Batting Avg: "+ str(battingAVG))
print("Gain/loss ratio: "+ ratio)
print("Average Gain: "+ str(avgGain))
print("Average Loss: "+ str(avgLoss))
print("Max Return: "+ maxR)
print("Max Loss: "+ maxL)
print("Total return over "+str(ng+nl)+ " trades: "+ str(totalR)+"%" )
print()


Results for MSFT going back to 2022-01-03 00:00:00, Sample size: 6 trades
EMAs used: [3, 5, 8, 10, 12, 15, 30, 35, 40, 45, 50, 60]
Batting Avg: 0.3333333333333333
Gain/loss ratio: 4.496493820720368
Average Gain: 15.353271368246357
Average Loss: -3.4144984915795265
Max Return: 29.885666899693163
Max Loss: 29.885666899693163
Total return over 6 trades: 28.91%

